# Machine Learning nos Exames de Bruxo de Hogwarts

### Descriçao

---
1. O objetivo deste notebook é aprender as 4 principais tarefas de ML (Classificação, Regressão, Agrupagamentos, Sistema de Recomendação)
2. Vamos construir um conjunto de dados baseado no universo do Harry Potter
3. O problema consiste em prever, baseado no histórico escolar do aluno, se ele está apto a entrar na casa que ele se candidatou.
---

### Dicionário


Fields	                                                  | Type  	  |    Description                              |
----------------------------------------------------------|:---------:|:-------------------------------------------:|
alunos 	  										  	  |string     | nome dos alunos	                    |
idade														  |integer    | idade dos alunos                         |
casas		     										  |string     | casa a qual ele se aplicou	                |
ano | int | ano em que se ele aplicou a vaga
ensino_medio | float | nota final do ensino médio
ensino_fundamental | float | nota final do ensino fundamental
label_alvo | string | resultado final da aplicacao



# Instalação dos pacotes

In [ ]:
!pip install pandas numpy scikit-learn keras Faker


# Documentação

1. ** Pandas ** -> [Link](https://pandas.pydata.org/docs/)
2. ** Numpy ** -> [Link](https://numpy.org/doc/)
3. ** Faker ** -> [Link](https://faker.readthedocs.io/en/master/)
4. ** Scikit Learn ** -> [Link](https://scikit-learn.org/stable/)
5. ** Keras ** -> [Link](https://keras.io/api/)
6. ** Tensor Flow ** -> [Tensor Flow](https://www.tensorflow.org/api_docs/python/tf/keras)
7. ** JobLib ** -> [JobLib](https://joblib.readthedocs.io/en/stable/)


# Importando as Bibliotecas

In [ ]:
import pandas as pd # para manipulacao de dados
import numpy as np # para manipulação de matrizes
import random #para gerar numeros aleatorios
from faker import Faker # para gerar nomes falsos

from sklearn.model_selection import train_test_split #para dividir os dados em teste e treino
from sklearn.preprocessing import StandardScaler, LabelEncoder# etapa de pre processamento de dados para deixar dados na mesma escala
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, silhouette_score #metricas de avaliacao
from sklearn.ensemble import RandomForestClassifier # algoritimo de classificação, baseado em arvore de decisão
from sklearn.linear_model import LogisticRegression # algoritimo de regressao, baseado em regressao logística
from sklearn.cluster import KMeans #para clusterizacao, agrupamento de dados
from sklearn.neighbors import NearestNeighbors #para o sistema de recomendação



import joblib #para salvar o modelos
import os #para criar uma pasta para hosperdarmos o modelo



# Construindo o Dataset Faker



In [ ]:

def generate(num): #recebe um valor especifico numerico que vai gerar a quantidade de linhas que precisamos
    fake = Faker() #crio uma variavel que vai funcionar como metodo para puxar a biblioteca
    data = []

    for i in range(num): #pecorre a quantidade de linhas que passamos no parametro
        aluno = fake.name() #gera nomes falsos
        idade = random.randint(11, 39) #gera uma idade entre 11 a 39 anos
        casas = random.choice(['Grifinória', 'Sonserina', 'Corvinal', 'Lufa-lufa']) #escolhe um item numa lista
        ano_de_ingresso = random.randint(2024 - idade, 2024) #gera um numero referente ao ano que vai entre 2024 e 2030
        nota_1 = round(random.uniform(0, 100), 2) # gera a nota do ensino fundamental
        nota_2 = 0 if nota_1 < 50 else round(random.uniform(0, 100), 2) # cria uma condicao para que se a nota_1 for menor que 50, automanticamente a nota_2 seja 0
        nota_final = nota_1 if nota_2 == 0 else round((nota_1 + nota_2) / 2, 2) #cria o valor da nota final que é a diferenca entre nota_1 - nota_2
        label_alvo = 'reprovado' if nota_final < 50 else 'aprovado' # verifica a nota final e cria condicao para aprovado ou reprovado

        row = {
            'aluno': aluno,
            'idade': idade,
            'casas': casas,
            'ano_de_ingresso': ano_de_ingresso,
            'ensino_fundamental': nota_1,
            'ensino_medio': nota_2,
            'nota_final': nota_final,
            'label_alvo': label_alvo
        }

        data.append(row)

    df = pd.DataFrame(data)
    return df

df = generate(1000)
df


# Classificação

#### **Objetivo**
1. Na classificação, nosso objetivo é prever ou descrever uma variavel categorica (string)
2. Vamos usar a coluna catégorica [label_alvo] para nosso estudo.
3. [Documentação dos algorítimos](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning)

In [ ]:
# Separar features e label
X = df.drop(['aluno', 'label_alvo', 'casas'], axis=1) #removendo as labels categoricas para treinamento
y = df['label_alvo']

In [ ]:
X
#y

In [ ]:
# Codificar variáveis categóricas
label_encoder = LabelEncoder()
#X['casas'] = label_encoder.fit_transform(X['casas'])
y = label_encoder.fit_transform(y) #transformando a categorica que vamos prever em numerica


In [ ]:
y

In [ ]:
#Dividir em teste e treino
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Inicializar e treinar o modelo
forest_classifier = RandomForestClassifier(random_state=42)
forest_classifier.fit(X_train, y_train)

In [ ]:
y_test

In [ ]:
y_pred = forest_classifier.predict(X_test) #prevendo o X_test

# Calcular a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred) #metrica de avaliacao

#print(f'O aluno {candidato} candidado a {casa_pretendida} está {predict}')
print(f'Acurácia do modelo: {accuracy:.2f}')


In [ ]:
X_test

#### Salvando o modelo

In [ ]:

folder = 'classificacao'

os.mkdir(folder)
with open(f'{folder}/harrypotter_random_forest.pkl', 'wb') as f:
    joblib.dump(forest_classifier, f)


### Reutilizando o modelo

In [ ]:
rf_classifier = joblib.load(f'{folder}/harrypotter_random_forest.pkl')

In [ ]:
#testando o modelo

nome_candidado = 'Maycon Batestin' #novo conjunto de dados
idade = 35
casa_pretendida = 'Lufa-lufa'
ano_de_ingresso = 2024
ensino_fundamental = 58.9
ensino_medio 	= 10.0
nota_final 	= (ensino_fundamental + ensino_medio) / 2

X_novo = [[idade, ano_de_ingresso, ensino_fundamental, ensino_medio, nota_final]]

y_pred = rf_classifier.predict(X_novo)
resultado = label_encoder.inverse_transform(y_pred)[0]

resposta = f"O aluno {nome_candidado} pretendido a casa {casa_pretendida} foi {resultado.upper()}"

In [ ]:
resposta

### salvando o modelo

# Regressão

#### **Objetivo**
1. Em regressão eu tento prever ou descrever uma categoria númerica (int)
2. Neste caso iremos criar uma regressão para podermos prever, com base nas notas final quais sao as chances do aluno de entrar na casa pretendida.
3. [Documentação dos algorítimos](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning)



In [ ]:
df = generate(1000)
df

In [ ]:
#transformando o label_alvo para uma variavel numerica

label_encoder = LabelEncoder()
df['label_alvo'] = label_encoder.fit_transform(df['label_alvo'])
df['casas'] = label_encoder.fit_transform(df['casas'])

In [ ]:
#Vamos separar nossos dados para um modelo supervisionado
#
X = df[['nota_final', 'casas']] # a variavel numerica que influencia o label_aprovado
y = df['label_alvo'] # a variavel alvo

In [ ]:
# dividindo em teste e treino
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#criando nosso modelo de regressao
regression_model = LogisticRegression()
regression_model.fit(X_train, y_train)


In [ ]:
#avaliando o modelo
score = regression_model.score(X_test, y_test)
print(f'Acurácia do modelo: {score}')



#### Salvando o modelo

In [ ]:
folder = 'regressao'

os.mkdir(folder)
with open(f'{folder}/harrypotter_regression_logistic.pkl', 'wb') as f:
    joblib.dump(regression_model, f)


### Reutilizando o modelo

In [ ]:
#importando ele
lg = joblib.load(f'{folder}/harrypotter_regression_logistic.pkl')

In [ ]:
aluno = 'Maycon Batestin'
casa_pretendida = 'Grifinória'

nota_final = 50
transfor_casa = label_encoder.transform([casa_pretendida])[0]
entrada = [[nota_final, transfor_casa]]
resultado = lg.predict_proba(entrada) # metrica para avaliar a estimativas de probabilidade, é uma array, onde o primeiro valor mais proximo do 1 representa a porcentagem em 100 e o segundo valor, mais proximo do zero indica a confianca



In [ ]:
print(f"Resultado final: {resultado}")
print(f"O aluno {aluno} tem {round(resultado[0][0],2) * 100}% de chance entrar na {casa_pretendida}")

# Agrupamentos (clustering)

#### **Objetivo**
1. Em agrupamentos eu não tento prever uma categoria especifica, tento agrupar os dados de acordo com caracteristicas em comum.
2. Neste nosso caso vamos criar um cluster para entender como os alunos realmente estão dividos entre si, pelas notas de ensino, pelo ano de ingresso, ou pela casa que se candidataram.
3. [Documentação dos algorítimos](https://scikit-learn.org/stable/modules/clustering.html#clustering)

In [ ]:
df = generate(1000)
df

In [ ]:
X = df.drop(['aluno'], axis=1)  # Remover as colunas 'aluno' e 'label_alvo'
X

In [ ]:
# Converter a coluna 'casas' em valores numéricos
le = LabelEncoder()
X['casas'] = le.fit_transform(X['casas'])
X['label_alvo'] = le.fit_transform(X['label_alvo'])

In [ ]:
# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Criar e treinar o modelo KMeans
km = KMeans(n_clusters=4, random_state=42)
clusters = km.fit_predict(X_scaled)

In [ ]:
# Adicionar os clusters ao DataFrame
df['grupo'] = clusters

In [ ]:
df

In [ ]:
df['grupo'].unique()

In [ ]:
df.loc[df['grupo'] == 0]

In [ ]:
# Calcular a inércia
inertia = km.inertia_
# Calcular o coeficiente de silhueta
silhouette = silhouette_score(X_scaled, clusters)

In [ ]:
print(f'Inércia: {inertia}')
print(f'Coeficiente de Silhueta: {silhouette}')

### Salvando o modelo

In [ ]:
folder = 'clustering'

os.mkdir(folder)
with open(f'{folder}/harrypotter_clustering.pkl', 'wb') as f:
    joblib.dump(km, f)


### Testando com novos dados

In [ ]:
#importando o modelo
km = joblib.load(f'{folder}/harrypotter_clustering.pkl')

In [ ]:
df_new = generate(10000) #gerando uma base de dados maior
X = df_new.drop(['aluno'], axis=1)  # Remover as colunas 'aluno' e 'label_alvo'

#o processo de pre processamento
le = LabelEncoder()
X['casas'] = le.fit_transform(X['casas'])
X['label_alvo'] = le.fit_transform(X['label_alvo'])

# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Criar e treinar o modelo KMeans
km = KMeans(n_clusters=4, random_state=42)
clusters = km.fit_predict(X_scaled)

df_new['grupo'] = clusters

inertia = km.inertia_
# Calcular o coeficiente de silhueta
silhouette = silhouette_score(X_scaled, clusters)

In [ ]:
print(f'Inércia: {inertia}')
print(f'Coeficiente de Silhueta: {silhouette}')

# Sistema de Recomendação

#### **Objetivo**
1.  Sistemas de recomendação são frequentemente implementados usando técnicas de aprendizado supervisionado, não supervisionado ou baseados em conteúdo.
2. Vamos recomendar a casa para o aluno, baseado na sua idade.
3. [Documentação dos algorítimos](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors)



In [ ]:
#criando nosso dataframe

df = generate(1000)
df

In [ ]:
df = df.loc[df['label_alvo'] == 'aprovado'] #do nosso dataset, vamos pegar apenas quem foi aprovado. Nao faz sentido recomendar para quem foi reprovado

In [ ]:
#selecionar apenas as variaveis que queremos

X = df[['idade', 'ano_de_ingresso']]
y = df['casas']

In [ ]:
# Transformando casas para um valor numerico

le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
#Treinando o sistema
vizinho = NearestNeighbors(algorithm = 'brute') #Algoritimo é um parametro usado para buscar os vizinhos de forma mais bruta
vizinho.fit(X, y)

#### Salvando o modelo

In [ ]:
folder = 'recomendacao'

os.mkdir(folder)
with open(f'{folder}/harrypotter_recomendacao.pkl', 'wb') as f:
    joblib.dump(vizinho, f)

#### Importando o modelo

In [ ]:
#importando o modelo
viz = joblib.load(f'{folder}/harrypotter_recomendacao.pkl')

####testando

In [ ]:
aluno = 'Maycon Batestin'
idade = 43
ano_de_ingresso = 2024

In [ ]:
# Criar o novo aluno
novo_aluno = pd.DataFrame({'idade': [idade], 'ano_de_ingresso': [ano_de_ingresso]})

In [ ]:
# Criar o modelo Nearest Neighbors
viz.fit(X, y)

In [ ]:
distancias, indices_vizinhos = vizinho.kneighbors(novo_aluno) # Encontrar os vizinhos mais próximos do novo aluno


In [ ]:
# Obter as casas dos vizinhos mais próximos
casas_vizinhos = df.iloc[indices_vizinhos[0]]['casas'].values


In [ ]:
# Encontrar a casa mais comum entre os vizinhos
casa_recomendada = pd.Series(casas_vizinhos).mode()[0] #aqui pegamos da serie casa_vizinhos que estabelecemos acima, e puxamos o primeiro item(0) para a variavel casa_recomendada


In [ ]:

print(f"Recomendamos para o {aluno} a casa {casa_recomendada.upper()}")